<a href="https://colab.research.google.com/github/rebeldose/newrepo/blob/master/classifying%203s%20and%207s.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!pip install -Uqq fastbook
import fastbook
fastbook.setup_book()

In [9]:

from fastai.vision.all import*
from fastbook import*
matplotlib.rc('image',cmap='Greys')

In [11]:
path = untar_data(URLs.MNIST_SAMPLE)

In [12]:
Path.BASE_PATH = path

In [13]:
path.ls()

(#3) [Path('labels.csv'),Path('valid'),Path('train')]

In [14]:
(path/'train').ls()

(#2) [Path('train/7'),Path('train/3')]

In [17]:
path.ls??


In [18]:
threes = (path/'train'/'3').ls().sorted()
sevens = (path/'train'/'7').ls().sorted()
threes

(#6131) [Path('train/3/10.png'),Path('train/3/10000.png'),Path('train/3/10011.png'),Path('train/3/10031.png'),Path('train/3/10034.png'),Path('train/3/10042.png'),Path('train/3/10052.png'),Path('train/3/1007.png'),Path('train/3/10074.png'),Path('train/3/10091.png')...]

In [22]:
Image.open(threes[1])

In [32]:
p=Image.open(sevens[8])
q=Image.open(sevens[2])
p

In [33]:
q

In [39]:
tensor(q)[4:10,4:10]

tensor([[  0,   0,   0,   0,   0,   0],
        [  0,   0,   0,   0,   0,   0],
        [  0,   0,   0,   0,   0,  13],
        [  0,  93, 185, 185, 185, 207],
        [  0, 253, 254, 254, 254, 253],
        [  0, 128, 254, 233, 103,  70]], dtype=torch.uint8)

In [ ]:
q_t = tensor(p)
df = pd.DataFrame(q_t)
df.style.set_properties(**{'font-size':'6pt'}).background_gradient('Greys')

In [46]:
sevens


(#6265) [Path('train/7/10002.png'),Path('train/7/1001.png'),Path('train/7/10014.png'),Path('train/7/10019.png'),Path('train/7/10039.png'),Path('train/7/10046.png'),Path('train/7/10050.png'),Path('train/7/10063.png'),Path('train/7/10077.png'),Path('train/7/10086.png')...]

In [53]:
seven_tensors = [tensor(Image.open(o)) for o in sevens]
three_tensors = [tensor(Image.open(o)) for o in threes]
len(seven_tensors),len(three_tensors)

(6265, 6131)

In [54]:
show_image(three_tensors[1])

In [55]:
three_tensors[1].shape

torch.Size([28, 28])

In [56]:
type(three_tensors)


list

In [57]:
stacked_sevens = torch.stack(seven_tensors).float()/255
stacked_threes = torch.stack(three_tensors).float()/255
stacked_threes.shape

torch.Size([6131, 28, 28])

In [59]:
len(stacked_sevens.shape)


3

In [64]:
stacked_threes.ndim

3

In [65]:
stacked_sevens.mean()

tensor(0.1145)

In [88]:
mean3 = stacked_threes.mean(0)
mean7 = stacked_sevens.mean(0)

In [77]:
type(mean3)

torch.Tensor

In [81]:
mean3.shape

torch.Size([28, 28])

In [ ]:
di = pd.DataFrame(mean3)
di.style.set_properties(**{'font-size':'6pt'}).background_gradient('Greys')

In [83]:
a_3 = stacked_threes[1]
show_image(a_3);

In [89]:
dist_3_abs = (a_3 - mean3).abs().mean()
dist_3_sqr = ((a_3 - mean3)**2).mean().sqrt()
dist_3_abs,dist_3_sqr

(tensor(0.1114), tensor(0.2021))

In [90]:
dist_7_abs = (a_3 - mean7).abs().mean()
dist_7_sqr = ((a_3 - mean7)**2).mean().sqrt()
dist_7_abs,dist_7_sqr

(tensor(0.1586), tensor(0.3021))

In [91]:
F.l1_loss(a_3.float(),mean7),F.mse_loss(a_3,mean7).sqrt()

(tensor(0.1586), tensor(0.3021))

In [92]:
valid_3_tens = torch.stack([tensor(Image.open(o)) for o in (path/'valid'/'3').ls()])
valid_3_tens = valid_3_tens.float()/255
valid_7_tens = torch.stack([tensor(Image.open(o)) for o in (path/'valid'/'7').ls()])
valid_7_tens = valid_7_tens.float()/255

In [93]:
valid_3_tens.shape,valid_7_tens.shape

(torch.Size([1010, 28, 28]), torch.Size([1028, 28, 28]))

In [99]:
def mnist_distance(a,b): return (a-b).abs().mean((-1,-2))

In [100]:
t = mnist_distance(a_3,mean3)
t

tensor(0.1114)

In [101]:
valid_3_dist = mnist_distance(valid_3_tens, mean3)
valid_3_dist, valid_3_dist.shape

(tensor([0.1471, 0.1136, 0.1289,  ..., 0.1058, 0.1413, 0.1206]),
 torch.Size([1010]))

In [102]:
def is_3(x):return mnist_distance(x,mean3) < mnist_distance(x,mean7)

In [106]:
is_3(a_3),is_3(a_3).float()

(tensor(True), tensor(1.))

In [107]:
is_3(valid_3_tens)

tensor([True, True, True,  ..., True, True, True])

In [109]:
accuracy_3s = is_3(valid_3_tens).float().mean()
accuracy_7s = (1-is_3(valid_7_tens).float()).mean()
accuracy_3s,accuracy_7s,(accuracy_3s+accuracy_7s)/2

(tensor(0.9168), tensor(0.9854), tensor(0.9511))